In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, ReLU, UpSampling2D
from keras.preprocessing import image

# 1. Histogram Equalization
def histogram_equalization(img):
    # Convert the image to HSV (Hue, Saturation, Value)
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Apply histogram equalization to the V (Value) channel (intensity)
    hsv_img[..., 2] = cv2.equalizeHist(hsv_img[..., 2])

    # Convert back to BGR color space
    eq_color_img = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2BGR)

    return eq_color_img

# 2. Gamma Correction
def gamma_correction(img, gamma):
    # Apply gamma correction
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in range(256)]).astype("uint8")
    gamma_corrected = cv2.LUT(img, table)
    return gamma_corrected

# 3. Deep Learning-Based Enhancement (CNN Model)
def create_deep_learning_model(input_shape=(500, 500, 3)):
    model = Sequential()

    # Convolution layers for feature extraction
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))

    # Upsampling layers to enhance image resolution
    model.add(UpSampling2D(size=(2, 2)))
    model.add(Conv2D(3, (3, 3), activation='sigmoid', padding='same'))  # Output layer with 3 channels for color image

    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def increase_contrast(img, factor=2):
    # Convert the image to float32 to avoid clipping
    img_float = img.astype(np.float32)

    # Increase the contrast by multiplying the pixel values by a factor
    img_float = img_float * factor

    # Clip the values to be within the valid range [0, 255]
    img_float = np.clip(img_float, 0, 255)

    # Convert back to uint8
    contrast_img = img_float.astype(np.uint8)

    return contrast_img

# Load your low-light image
good_img = cv2.imread('2.png')  # Replace with the path to your image
ori_img = cv2.imread('2L.png')  # Replace with the path to your image

# #image with high conrast
high_contrast_img = increase_contrast(ori_img)

# Apply Histogram Equalization
hist_eq_img = histogram_equalization(ori_img)

# Apply Gamma Correction
gamma_img = gamma_correction(high_contrast_img, gamma=2)  # You can change the gamma value for desired brightness

# Show the results
plt.figure(figsize=(15, 5))

plt.subplot(1, 4, 1)
plt.imshow(cv2.cvtColor(ori_img, cv2.COLOR_BGR2RGB))
plt.title("Original Image")

plt.subplot(1, 4, 2)
plt.imshow(cv2.cvtColor(good_img, cv2.COLOR_BGR2RGB))
plt.title("Good Image")

plt.subplot(1, 4, 3)
plt.imshow(cv2.cvtColor(hist_eq_img, cv2.COLOR_BGR2RGB))
plt.title("Histogram Equalized Image")

plt.subplot(1, 4, 4)
plt.imshow(cv2.cvtColor(gamma_img, cv2.COLOR_BGR2RGB))
plt.title("Gamma Corrected Image")

plt.show()

# Now, we will prepare the image for deep learning enhancement
# Resize image to 500x500
img_resized = cv2.resize(ori_img, (500, 500))

# Create and train the deep learning model
model = create_deep_learning_model()

# Note: Training the model would require a dataset of low-light and corresponding high-quality images.
# For demonstration, we assume the model is pre-trained or fine-tuned.

# For this example, we'll simulate a "fake" training by enhancing the image.
# Normally, you'd use a dataset with both low-light and high-quality images to train the model.

enhanced_img = model.predict(np.expand_dims(np.float32(img_resized) / 255.0, axis=0))

# Display the enhanced image from the deep learning model
plt.imshow(enhanced_img[0])  # Assuming the model outputs an image in the range [0, 1]
plt.title("Deep Learning Enhanced Image")
plt.show()


: 

In [26]:
import cv2
import numpy as np

def calculate_psnr(original, enhanced):
    # Resize images to the same shape (e.g., 256x256 or 512x512)
    size = (256, 256)  # You can change the size to any desired resolution
    original_resized = cv2.resize(original, size)
    enhanced_resized = cv2.resize(enhanced, size)

    mse = np.mean((original_resized - enhanced_resized) ** 2)
    if mse == 0:
        return 100  # No noise, perfect similarity

    psnr = 20 * np.log10(255.0 / np.sqrt(mse))
    return psnr

# Example of how to call the function:
psnr_value = calculate_psnr(img, hist_eq_img)
print(f"PSNR: {psnr_value} dB")


PSNR: 27.200367642400252 dB


In [28]:
from skimage.metrics import structural_similarity as ssim
import math

# Method to calculate PSNR (Peak Signal-to-Noise Ratio)
def calculate_psnr(original, enhanced):
    mse = np.mean((original - enhanced) ** 2)
    if mse == 0:
        return 100  # No noise, perfect similarity
    max_pixel = 255.0  # For 8-bit image
    psnr = 20 * math.log10(max_pixel / math.sqrt(mse))
    return psnr

# Method to calculate SSIM (Structural Similarity Index)
def calculate_ssim(original, enhanced):
    ssim_value = ssim(original, enhanced, multichannel=True)
    return ssim_value

# Method to visualize the improvement using a graph
def visualize_improvement(original, enhanced):
    # Calculate PSNR and SSIM for comparison
    psnr_value = calculate_psnr(original, enhanced)
    ssim_value = calculate_ssim(original, enhanced)

    print(f"PSNR: {psnr_value} dB")
    print(f"SSIM: {ssim_value}")

    # Plot the original and enhanced images side by side
    plt.figure(figsize=(15, 5))

    # Original Image
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    plt.title("Original (Low-light Image)")
    plt.axis('off')

    # Enhanced Image
    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(enhanced, cv2.COLOR_BGR2RGB))
    plt.title("Enhanced Image")
    plt.axis('off')

    # Show PSNR and SSIM on the graph
    plt.figtext(0.5, 0.01, f'PSNR: {psnr_value:.2f} dB, SSIM: {ssim_value:.3f}', ha='center', va='center', fontsize=12, color='black')

    plt.show()

# Example of using these methods after enhancement
# Apply Histogram Equalization and Gamma Correction
hist_eq_img = histogram_equalization(img)
# gamma_img = gamma_correction(img, gamma=1.5)

# Assuming the deep learning model has already been trained or pre-loaded
# We'll use the model to enhance the image
enhanced_img = model.predict(np.expand_dims(np.float32(img_resized) / 255.0, axis=0))

# Visualize and evaluate the enhancement
visualize_improvement(img, enhanced_img[0])  # Assuming model output is in range [0, 1]


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 936ms/step


ValueError: win_size exceeds image extent. Either ensure that your images are at least 7x7; or pass win_size explicitly in the function call, with an odd value less than or equal to the smaller side of your images. If your images are multichannel (with color channels), set channel_axis to the axis number corresponding to the channels.